In [1]:
import tensorflow as tf

In [2]:
import numpy as np

## 1. 损失函数

即预测值与已知答案之间的差距。神经网络模型的效果及优化的目标是通过损失函数来定义的。

损失函数是表示神经网络性能的“恶劣程度”的指标，即当前的神经网络对监督数据在多大程度上不拟合，在多大程度上不一致。而我们神经网络的优化目标就是使得损失函数达到最小，即性能的“恶劣程度”达到最小。

损失函数可以使用任意函数，而比较常用的损失函数有交叉熵误差（cross entropy error，CEE）和均方误差（Mean Squared Error, MSE）。

### 1.1 交叉熵误差

表征的是两个概率分布之间的距离，交叉熵越大说明两个概率分布越远，交叉熵越小说明两个概率分布分布越近，是分类问题中使用较广泛的损失函数。 交叉熵误差损失函数定义如下：

$$H(t, y)=-\sum_{i=1}^{n} t_i \cdot ln y_i$$

其中 $t_i$ 为第 $i$ 个数据的真实值，$y_i$ 为神经网络对第 $i$ 个数据的预测值。

对于多分类问题，神经网络的输出一般不是概率分布，因此需要引入softmax层，使得输出服从概率分布。

TensorFlow API：
 + `tf.keras.losses.categorical_crossentropy `
 + `tf.nn.softmax_cross_entropy_with_logits `
 + `tf.nn.sparse_softmax_cross_entropy_with_logits `
 
也可以直接利用公式 `h = lambda t, y: -tf.reduce_sum(t * np.log(y))` 来计算。

**e.g.** 对于二分类问题，已知答案 `t=(1,0)`，现有预测值 `y1=(0.6, 0.4)` 和 `y2=(0.8, 0.2)`，哪个更接近标准答案？

In [3]:
t = np.array([1, 0])
y1 = np.array([0.6, 0.4])
y2 = np.array([0.8, 0.2])

In [4]:
# 使用自己定义的交叉熵误差损失函数
cee = lambda t, y: -tf.reduce_sum(t * np.log(y))
h1 = cee(t, y1)
h2 = cee(t, y2)
print('h1:', h1)
print('h2:', h2)
'h1' if h1 > h2 else 'h2'

h1: tf.Tensor(0.5108256237659907, shape=(), dtype=float64)
h2: tf.Tensor(0.2231435513142097, shape=(), dtype=float64)


'h1'

In [5]:
# 使用 TensorFlow API
tfH1 = tf.losses.categorical_crossentropy(t, y1)
tfH2 = tf.losses.categorical_crossentropy(t, y2)
print('tfH1:', tfH1)
print('tfH2:', tfH2)
'tfH1' if tfH1 > tfH2 else 'tfH2'

tfH1: tf.Tensor(0.5108256237659907, shape=(), dtype=float64)
tfH2: tf.Tensor(0.2231435513142097, shape=(), dtype=float64)


'tfH1'

对比之下验证了使用计算公式和使用 API 的计算结果是一致的

#### 1.1.1 softmax 与交叉熵误损失函数结合

在执行分类任务时，通常先用 softmax 函数，让输出结果符合概率分布，再求交叉熵损失函数。

TensorFlow 给出了可以同时计算概率分布和交叉熵的函数：
 + `tf.nn.softmax_cross_entropy_with_logits`

In [6]:
label = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0]])
output = np.array([[12, 3, 2], [3, 10, 1], [1, 2, 5], [4, 6.5, 1.2], [3, 6, 1]])

In [7]:
# 分步计算 softmax 和 cross entropy
y_ = tf.nn.softmax(output)
tf.losses.categorical_crossentropy(label, y_)

<tf.Tensor: shape=(5,), dtype=float64, numpy=
array([1.68795487e-04, 1.03475622e-03, 6.58839038e-02, 2.58349207e+00,
       5.49852354e-02])>

In [8]:
# 结合计算
tf.nn.softmax_cross_entropy_with_logits(label, output)

<tf.Tensor: shape=(5,), dtype=float64, numpy=
array([1.68795487e-04, 1.03475622e-03, 6.58839038e-02, 2.58349207e+00,
       5.49852354e-02])>

可见两种方式的结果一致。

### 1.2 均方误差

均方误差（Mean Square Error）是回归问题最常用的损失函数。回归问题解决的是对具体数值的预测，比如房价预测、销量预测等。这些问题需要预测的不是一个事先定义好的类别，而是一个任意实数。均方误差损失函数定义如下：

$$MSE(t,y)=\frac{ \sum_{i=1}^{n}(t_i - y_i)^2}{n}$$

其中 $t_i$ 为第 $i$ 个数据的真实值，$y_i$ 为神经网络的对第 $i$ 个数据的预测值。

TensorFlow API:
 + `tf.keras.losses.MSE`

也可以直接利用公式 `mse = lambda t, y: tf.reduce_sum(tf.square(y - t))` 来计算。

**e.g.** 预测酸奶的日销量 $y$ 和影响日销量的因素 $x_1$ 和 $x_2$ 的关系。

在建模之前，应预先采集的数据有：酸奶的日销量 $y$ 和影响日销量的因素 $x_1$ 和 $x_2$ ，数据量越大越好（最佳情况是：酸奶的产量=酸奶的销售量）

**由于手头没有相关数据**，所以我们自行制造一套数据集：随机生成 $x_1$ 和 $x_2$， 标准答案：$y = x1 + x2$，噪声：$-0.05 \sim +0.05$。拟合可以预测销量的函数。
|
>这里的意思是，先通过随机数和一个关系来制造数据，并引入一定的噪声使得不完全相等，然后使用梯度下降法来预测数据之间的关系，并与我们给定的标准答案相比较，最终目的就是使得预测值与标准答案之间的损失函数值尽可能的小。

In [9]:
# 制造数据集
SEED = 23455
# 生成 [0, 1) 之间的随机数
rdm = np.random.RandomState(seed=SEED)
# 生成 32 组数据，第 0 列表示 x1 的数据，第 1 列表示 x2 的数据
x = rdm.rand(32, 2)
# 噪声 [0,1) / 10 -0.05 ==> [0, 0.1) - 0.05 ==> [-0.05, 0.05)
noise = rdm.rand() / 10 - 0.05
# 生成标签
t = [[x1 + x2 + noise] for (x1, x2) in x]
x = tf.cast(x, dtype=tf.float32)

定义 MSE 损失函数

In [10]:
mse = lambda t, y: tf.reduce_sum(tf.square(t - y))

定义训练函数

In [11]:
# x 为输入值，t 为正确结果
def train(x, t, lr=0.002, epoch=10000, loss_function=mse):
    # 随机初始化权重参数，设定随机种子是为了做对比，实际情况无需设定该值
    w1 = tf.Variable(tf.random.normal([2, 1], stddev=1, seed=1))
    for i in range(epoch):
        with tf.GradientTape() as tape:
            y = tf.matmul(x, w1)
            loss = loss_function(t, y)
        grads = tape.gradient(loss, w1)
        w1.assign_sub(lr * grads)

        if i % 500 == 0:
            print('After {} training steps, w1 is {}'.format(i, w1.numpy()))
    print('Final w1 is:', w1.numpy())

In [12]:
train(x, t)

After 0 training steps, w1 is [[-0.7559013]
 [ 1.5153551]]
After 500 training steps, w1 is [[1.0181998]
 [1.0180802]]
After 1000 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 1500 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 2000 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 2500 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 3000 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 3500 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 4000 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 4500 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 5000 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 5500 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 6000 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 6500 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 7000 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 7500 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 8000 training steps, w1 is [[1.02031

根据拟合结果，可知 $y \approx x_1 + x_2$，与自定义的标准答案 $y = x_1 + x_2$ 基本一致，说明预测酸奶的销量公式拟合正确。

### 1.3 自定义损失函数

根据具体任务和目的，可设计不同的损失函数。根据上面关于酸奶销售的例子可以看出，损失函数的定义能极大的影响模型预测效果，好的损失函数设计对于模型训练能够起到良好的引导作用。

其实，上面的例子使用均方误差作为损失函数，默认认为酸奶销量预测的多了或者少了，其损失是一样的。然而真实情况是，酸奶的销售量预测多了，损失的是成本，预测少了损失的是利润，而利润和成本往往不相等。在这种情况下，使用均方误差来计算损失函数是没办法让利益最大化的，这时我们可以使用自定义损失函数，使用自定义损失函数计算每一个预测结果 $y$ 与标准答案 $t$ 产生的损失累计和：

$$loss(t, y)=\sum_{n}f(t, y)$$

其中 $t$ 为正确结果， $y$ 为预测值。根据分析，设计出对于酸奶销售量预测的自定义损失函数如下：

$$f(t, y)=\left\{\begin{matrix}
PROFIT \cdot (t - y)) & y < t & 预测的y少了，损失利润（PROFIT）\\ 
COST \cdot (y - t) & y \geq t & 预测的y多了，损失成本（COST）
\end{matrix}\right.$$

自定义损失函数的代码表示：`loss_custom = lambda t, y: tf.reduce_sum(tf.where(tf.greater(t, y), PROFIT * (t - y), COST * (y - t)))`

举例来说明自定义损失函数的含义：假设一瓶酸奶的成本（COST）为 1 元，一瓶酸奶的利润（PROFIT）为 99 元
 + 如果酸奶销量预测少了，则损失了`少的销量*单位利润99元`
 + 如果酸奶销量预测多了，则损失了`多的销量*单位成本1元`
 
根据上述情况可以看出，预测少了损失更大，因此希望生成的预测函数（即销量 $y$ 与影响因素 $x_1$ 和 $ x_2$ 的关系）往多了预测。

相比于之前完成的代码，只需要更改损失函数即可

In [13]:
PROFIT, COST = 99, 1
loss_custom = lambda t, y: tf.reduce_sum(tf.where(tf.greater(t, y), PROFIT * (t - y), COST * (y - t)))

In [14]:
train(x, t, loss_function=loss_custom)

After 0 training steps, w1 is [[2.7448583]
 [3.1585946]]
After 500 training steps, w1 is [[1.2346897]
 [1.2794034]]
After 1000 training steps, w1 is [[1.0419035]
 [1.3938723]]
After 1500 training steps, w1 is [[1.1125113]
 [1.2793158]]
After 2000 training steps, w1 is [[1.0685171]
 [1.0530982]]
After 2500 training steps, w1 is [[1.1136303]
 [1.4584029]]
After 3000 training steps, w1 is [[1.0555954]
 [1.2353483]]
After 3500 training steps, w1 is [[1.0605626]
 [1.1579232]]
After 4000 training steps, w1 is [[1.0143902]
 [1.3628445]]
After 4500 training steps, w1 is [[1.0887334]
 [1.0147449]]
After 5000 training steps, w1 is [[1.0664105]
 [1.119    ]]
After 5500 training steps, w1 is [[1.0919956]
 [1.078778 ]]
After 6000 training steps, w1 is [[1.1625124]
 [1.2568647]]
After 6500 training steps, w1 is [[1.1187032]
 [1.2894193]]
After 7000 training steps, w1 is [[1.0036174]
 [1.4984071]]
After 7500 training steps, w1 is [[1.0576496]
 [1.0948691]]
After 8000 training steps, w1 is [[1.1071676

可以发现拟合结果对于影响销量的两个因素 $x_1$ 和 $x_2$ 的系数都偏大，模型的确在尽量往多了预测，符合原本预期。

如果我们把一瓶酸奶的成本改为 99 元，一瓶酸奶的利润改为 1 元，再测试一下拟合结果

In [15]:
PROFIT, COST = 1, 99
train(x, t, loss_function=loss_custom)

After 0 training steps, w1 is [[-0.17841518]
 [ 0.03795246]]
After 500 training steps, w1 is [[0.9281722]
 [0.5942146]]
After 1000 training steps, w1 is [[0.933755  ]
 [0.39487356]]
After 1500 training steps, w1 is [[-1.0699775]
 [-1.210458 ]]
After 2000 training steps, w1 is [[0.5103349]
 [1.0495176]]
After 2500 training steps, w1 is [[0.91663736]
 [0.92462695]]
After 3000 training steps, w1 is [[ 0.25667965]
 [-0.5379779 ]]
After 3500 training steps, w1 is [[0.96828336]
 [0.642376  ]]
After 4000 training steps, w1 is [[ 0.7433436 ]
 [-0.01640141]]
After 4500 training steps, w1 is [[0.97669476]
 [0.88775206]]
After 5000 training steps, w1 is [[1.0474858 ]
 [0.56453145]]
After 5500 training steps, w1 is [[0.46987772]
 [0.11492041]]
After 6000 training steps, w1 is [[-0.2426675]
 [-0.413855 ]]
After 6500 training steps, w1 is [[-0.1491549]
 [-1.1487912]]
After 7000 training steps, w1 is [[-0.49467975]
 [-0.8647992 ]]
After 7500 training steps, w1 is [[ 0.8402844 ]
 [-0.16282651]]
After 

可以发现拟合结果对于影响销量的两个因素 $x_1$ 和 $x_2$ 的系数相比之前都偏小，模型在尽量把销量往少了了预测。